# 法一：明细统计
* 历史近n天汇总明细（90天）

In [1]:
from impala.dbapi import connect
from impala.util import as_pandas
import pandas as pd
import numpy as np
import requests
import warnings
import pymysql
import json
import ast
import re
import random
import seaborn as sns
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
tqdm.pandas(desc="Processing:")
warnings.filterwarnings("ignore")

In [2]:
def get_imageid(image_ids):  
    """ 素材名：提取 """
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)
    
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name,
            label_ids
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND image_name like '%QMZZ%'
    '''
    cur.execute(sql.format(image_ids))
    result = pd.read_sql(sql.format(image_ids), conn)
    # 关闭链接
    cur.close()
    conn.close()

    return result

In [3]:
conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
sql = '''
      SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL
      '''
cur.execute(sql)
game_ids = cur.fetchall()
game_ids = list(map(lambda x: x['game_id'], game_ids))
game_ids = [str(i) for i in game_ids]
game_ids = ','.join(game_ids)

## 分大标签

### 小标签映射

In [5]:
mapping = pd.read_excel('./data/label.xlsx', header=None)
mapping = mapping.loc[:, :1]
mapp = pd.DataFrame()
for key_word in ['3渲2基建战斗','3D基建战斗','大基建','对比','剧情','兵种','录屏','小游戏','泛素材']:
    mapp = mapp.append(mapping[mapping.iloc[:,1].str.contains(key_word)])
mapp

,0,1
27,220,QMZZ：3渲2基建战斗
4,117,QMZZ：3D基建战斗
25,201,QMZZ：大基建-N
1,114,QMZZ：对比类
2,115,QMZZ：剧情（主漂流荒岛）
13,148,QMZZ：海岛剧情
28,221,QMZZ：兵种
17,159,QMZZ：录屏
6,119,QMZZ：小游戏
16,157,QMZZ：小游戏


In [6]:
label_ids = list(map(lambda x: x, mapp[0]))
label_ids = [str(i) for i in label_ids]
label_ids = ','.join(label_ids)
label_ids

'220,117,201,114,115,148,221,159,119,157,196,116,253'

### 小标签明细

In [7]:
def label_detail(game_ids, label_ids):
    """ 明细：标签维度 """
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                        passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    sql = '''
        /*手动查询*/
        SELECT
            c.image_id,
            d.image_name,
            d.label_ids,
            c.amount,
            c.create_role_num,
            c.new_role_money,
            c.sum_money
        FROM
            (SELECT
                b.image_id,
                IFNULL(sum(a.amount),0) as amount,
                IFNULL(sum(a.create_role_num),0) as create_role_num,
                IFNULL(sum(a.new_role_money),0) as new_role_money,
                IFNULL(sum(a.sum_money),0) as sum_money
            FROM
                db_stdata.st_lauch_report a LEFT JOIN db_data_ptom.ptom_plan b ON a.channel_id = b.chl_user_id 
                AND a.source_id = b.source_id
            WHERE
                a.tdate>=date( NOW() - INTERVAL 2160 HOUR )  ## 消耗统计3个月内
                AND b.platform=1
                AND b.game_id IN ({})
                AND a.tdate_type='day'
                AND a.media_id=10
            GROUP BY
                b.image_id) c LEFT JOIN db_data_ptom.ptom_image_info d ON c.image_id = d.image_id
        WHERE
            d.label_ids IN ({})
    '''
    res1 = pd.read_sql(sql.format(game_ids, label_ids), conn)
    res1.loc[(res1['label_ids'] == '') | (res1['label_ids'] == -1), 'label_ids'] = np.nan
    res1.dropna(subset=['image_id','label_ids'], inplace=True)
    res1['image_id'] = res1['image_id'].astype(int)
    
    return res1

In [8]:
df = label_detail(game_ids, label_ids)

In [9]:
df = df[df['amount'] > 0]  # 有消耗

In [10]:
print(df.shape)
df.head()

(465, 7)


,image_id,image_name,label_ids,amount,create_role_num,new_role_money,sum_money
23,29475,QMZZ2253-竖版ly（3D复用3渲2）（MOD）（TT）,220,2100.56,7.0,0.0,1.0
26,29642,QMZZ-2040竖版ly（TT）,253,20621.22,108.0,23.0,774.0
40,30293,QMZZ442-横版CHL（三渲二）（TT）,115,641.09,3.0,0.0,168116.0
54,31031,QMZZ540-横版xy（无帝国）（3D复用3渲2）（TT）,148,601.80,0.0,0.0,1496.0
88,32124,QMZZ748-竖版Z（3D复用3渲2）（TT）,148,28753.63,183.0,25.0,86.0


### 大标签映射

In [11]:
label_mapping = {'220':1,'117':2,'201':3,'114':4,'148':5,'115':5,'221':6,'159':7,'119':8,'157':8,'196':8,'116':9,'253':9}

In [17]:
def label_big(label_mapping, data):
    """ 大标签映射 """
    if len(data['label_ids']) == 3:
        try:
            data['label_ids'] = label_mapping[data['label_ids']]
            return pd.DataFrame(data.to_frame().values.T, columns=data.to_frame().index)
        except:
            return pd.DataFrame(columns=data.to_frame().index)
    
    df = pd.DataFrame()
    count = []
    for label in data['label_ids'].split(','):
        try:
            big = label_mapping[label]
            if big not in count:
                count += [big]
                data['label_ids'] = big
                df = df.append(pd.DataFrame(data.to_frame().values.T, columns=data.to_frame().index))
        except:
            continue
    return df

In [28]:
a = df.progress_apply(lambda x: label_big(label_mapping, x), axis=1)

In [13]:
df_1 = pd.concat(a.tolist(), axis=0)

In [14]:
print(df_1.shape)
df_1.head()

(489, 7)


,image_id,image_name,label_ids,amount,create_role_num,new_role_money,sum_money
0,28867,QMZZ137-横版xy,2,1038.02,6,0,0
0,29475,QMZZ2253-竖版ly（3D复用3渲2）（MOD）（TT）,1,2100.56,7,0,1
0,29642,QMZZ-2040竖版ly（TT）,9,20621.2,108,23,774
0,30293,QMZZ442-横版CHL（三渲二）（TT）,5,641.09,3,0,157376
0,31031,QMZZ540-横版xy（无帝国）（3D复用3渲2）（TT）,5,601.8,0,0,1240


In [15]:
df_1['image_id'].value_counts()

43856    3
42439    2
42339    2
42268    2
34785    2
        ..
37512    1
42518    1
42636    1
43164    1
31745    1
Name: image_id, Length: 458, dtype: int64

### 大标签分组

In [16]:
df_1 = df_1.groupby('label_ids')[['amount','create_role_num','new_role_money','sum_money']].sum()
df_1.reset_index(inplace=True)

In [17]:
df_1['roi'] = df_1['sum_money'] / df_1['amount']
df_1

,label_ids,amount,create_role_num,new_role_money,sum_money,roi
0,1,9803128.00,63817.0,95745.0,8011848.0,0.817275
1,2,2364045.46,16714.0,36971.0,1153367.0,0.487879
2,3,130875.34,888.0,3326.0,304733.0,2.328422
3,4,79405.25,508.0,990.0,22742.0,0.286404
4,5,1506531.06,9931.0,12637.0,2009303.0,1.333728
5,6,409414.22,3480.0,8125.0,930222.0,2.272080
6,7,60906.82,426.0,632.0,261607.0,4.295200
7,8,582904.84,5724.0,7331.0,220402.0,0.378110
8,9,1271389.53,10738.0,14075.0,906742.0,0.713190


In [18]:
mapp['label_ids'] = list(map(lambda x: label_mapping[str(x)], mapp[0]))

In [19]:
pd.merge(df_1, mapp, on=['label_ids'])

,label_ids,amount,create_role_num,new_role_money,sum_money,roi,0,1
0,1,9803128.00,63817.0,95745.0,8011848.0,0.817275,220,QMZZ：3渲2基建战斗
1,2,2364045.46,16714.0,36971.0,1153367.0,0.487879,117,QMZZ：3D基建战斗
2,3,130875.34,888.0,3326.0,304733.0,2.328422,201,QMZZ：大基建-N
3,4,79405.25,508.0,990.0,22742.0,0.286404,114,QMZZ：对比类
4,5,1506531.06,9931.0,12637.0,2009303.0,1.333728,115,QMZZ：剧情（主漂流荒岛）
5,5,1506531.06,9931.0,12637.0,2009303.0,1.333728,148,QMZZ：海岛剧情
6,6,409414.22,3480.0,8125.0,930222.0,2.272080,221,QMZZ：兵种
7,7,60906.82,426.0,632.0,261607.0,4.295200,159,QMZZ：录屏
8,8,582904.84,5724.0,7331.0,220402.0,0.378110,119,QMZZ：小游戏
9,8,582904.84,5724.0,7331.0,220402.0,0.378110,157,QMZZ：小游戏


# 法二：明细统计
* 素材评分窗口历史明细（90天）

## 数据加载

In [20]:
train_data = pd.read_csv('./data/train_data_1112.csv')
train_data.columns  # 未来30日的'image_create_role_roi'

Index(['image_id', 'image_name', 'media_id', 'image_launch_time',
       'image_source_total_num', 'image_run_date_amount',
       'image_create_role_pay_num', 'image_create_role_num',
       'image_create_role_pay_sum', 'image_source_num',
       'image_create_role_pay_rate', 'image_create_role_cost',
       'image_create_role_pay_cost', 'image_valid_source_num',
       'image_valid_source_rate', 'image_pay_sum_ability',
       'image_pay_num_ability', 'image_create_role_roi',
       'image_create_role_retain_1d', 'create_role_30_pay_sum',
       'model_run_datetime', 'data_win'],
      dtype='object')

In [21]:
train_data = train_data[train_data['media_id'] == 10]  # 1、过滤头条  # TODO
# 2、过滤违规
def illegal_image():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)

    sql = 'select image_id from db_data_ptom.ptom_image_info where FIND_IN_SET(1001706, game_ids)'
    cur.execute(sql)
    result = pd.read_sql(sql, conn)
    # 关闭链接
    cur.close()
    conn.close()
    return result['image_id'].values

train_data = train_data[~train_data['image_id'].isin(illegal_image())]

### 小标签映射

In [22]:
def label_extract():
    """ 小标签 """
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                        passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    sql = '''
        /*手动查询*/
        SELECT
            image_id,
            label_ids
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            game_ids IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL )
            AND create_time >= date( NOW() - INTERVAL 10000 HOUR )  ## 创建时间1个月内
    '''
    res1 = pd.read_sql(sql, conn)
    return res1

In [23]:
train_data = pd.merge(train_data, label_extract(), on=['image_id'], how='left')
train_data.loc[(train_data['label_ids'] == '') | (train_data['label_ids'] == -1), 'label_ids'] = np.nan
train_data.dropna(subset=['image_id','label_ids'], inplace=True)
train_data['image_id'] = train_data['image_id'].astype(int)

### 大标签映射

In [24]:
label_mapping = {'220':1,'117':2,'201':3,'114':4,'148':5,'115':5,'221':6,'159':7,'119':8,'157':8,'196':8,'116':9,'253':9}

In [25]:
def label_big(label_mapping, data):
    """ 大标签映射 """
    if len(data['label_ids']) == 3:
        try:
            data['label_ids'] = label_mapping[data['label_ids']]
            return pd.DataFrame(data.to_frame().values.T, columns=data.to_frame().index)
        except:
            return pd.DataFrame(columns=data.to_frame().index)
    
    df = pd.DataFrame()
    count = []
    for label in data['label_ids'].split(','):
        try:
            big = label_mapping[label]
            if big not in count:
                count += [big]
                data['label_ids'] = big
                df = df.append(pd.DataFrame(data.to_frame().values.T, columns=data.to_frame().index))
        except:
            continue
    return df

In [26]:
a = train_data.progress_apply(lambda x: label_big(label_mapping, x), axis=1)

Processing:: 100%|██████████| 32503/32503 [00:32<00:00, 988.11it/s] 


In [27]:
df_2 = pd.concat(a.tolist(), axis=0)
# 去重
df_2.drop_duplicates(subset=['image_id','label_ids'], keep='last', inplace=True)

In [28]:
print(df_2.shape)
df_2.head()

(629, 23)


,image_id,image_name,media_id,image_launch_time,image_source_total_num,image_run_date_amount,image_create_role_pay_num,image_create_role_num,image_create_role_pay_sum,image_source_num,...,image_valid_source_num,image_valid_source_rate,image_pay_sum_ability,image_pay_num_ability,image_create_role_roi,image_create_role_retain_1d,create_role_30_pay_sum,model_run_datetime,data_win,label_ids
0,29284,QMZZ223-横版xy,10,2021-01-27,1,110.02,0,0,0,1,...,0,0,0,0,0,NaN,NaN,2021-02-01,3,9
0,29288,QMZZ225-横版xy,10,2021-01-28,4,595.45,0,2,0,2,...,0,0,0,0,0,0.3333,0,2021-02-01,3,9
0,29475,QMZZ253-横版LL,10,2021-02-02,7,484.11,0,7,0,1,...,0,0,0,0,0,0.0714,0,2021-02-11,3,1
0,29867,QMZZ294-竖版xy（3D混剪）,10,2021-02-09,5,353.02,0,0,0,5,...,2,0.4,0,0,0,0.2308,204,2021-02-17,3,5
0,29914,QMZZ303-横版xy（3D）,10,2021-02-10,6,224.5,0,1,0,4,...,0,0,0,0,0,0,0,2021-02-18,3,5


### 大标签分组

In [29]:
df_2 = df_2.groupby('label_ids')[['create_role_30_pay_sum']].sum()
df_2.reset_index(inplace=True)

In [30]:
pd.merge(df_2, mapp, on=['label_ids'])

,label_ids,create_role_30_pay_sum,0,1
0,1,221369.0,220,QMZZ：3渲2基建战斗
1,2,45159.0,117,QMZZ：3D基建战斗
2,3,7628.0,201,QMZZ：大基建-N
3,4,439.0,114,QMZZ：对比类
4,5,25885.0,115,QMZZ：剧情（主漂流荒岛）
5,5,25885.0,148,QMZZ：海岛剧情
6,6,2531.0,221,QMZZ：兵种
7,7,6183.0,159,QMZZ：录屏
8,8,5114.0,119,QMZZ：小游戏
9,8,5114.0,157,QMZZ：小游戏


# 规则加权
* 权重 == 法一ROI * 法二ROI * 人工比例系数w
* 3渲2基建、3D基建、大基建N、对比、荒岛剧情、兵种、录屏、小游戏、泛素材

In [31]:
label_mapping = {'220':1,'117':2,'201':3,'114':4,'148':5,'115':5,'221':6,'159':7,'119':8,'157':8,'196':8,'116':9,'253':9}

## 标签分配

In [32]:
def allocate(df1, df2, w, num):
    """ 分配：大标签获取 """
    weight = df1['roi'] * df2['create_role_30_pay_sum'] * w
    def normalization(data):
        _range = np.max(data) - np.min(data)
        return (data - np.min(data)) / _range
    
    weight = normalization(weight.astype(int))
    seq = pd.Series([1,2,3,4,5,6,7,8,9]).sample(n=num, replace=True, weights=weight).values
    print(seq)
    return seq

## **选择**
* 素材选择

In [33]:
class ImageSec():
    """ 素材类 """
    def __init__(self, mapping, flex_coe, plan_num):
        self.mapping = mapping    # 大小标签映射关系Dict
        self.flex_coe = flex_coe  # 人工系数Series
        self.plan_num = plan_num  # 计划数量Int
        assert len(set(self.mapping.values())) == len(self.flex_coe), "维度差异！"
    
    @staticmethod
    def get_images_candi(labels, mapping):
        """ 标签：素材下钻（评分） """
        def collect(labels_str, media_id, score, start, end):
            """ 评分筛选 """
            conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hive',
                           password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
            cursor = conn.cursor()
            sql_engine = 'set hive.execution.engine=tez'
            sql_queque = 'set tez.queue.name=offline'
            sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id={} and score>={} and label_ids in ({}) and dt>=date_sub(current_date, {}) and dt<=date_sub(current_date, {}) group by image_id,label_ids'
            cursor.execute(sql_engine)
            cursor.execute(sql_queque)
            cursor.execute(sql.format(media_id, score, labels_str, start, end))
            result = as_pandas(cursor)
            # 关闭链接
            cursor.close()
            conn.close()
            return result['image_id'].values.tolist()
        
        candi = {}  # 候选素材池（分标签）
        for label in tqdm(set(labels)):  # 标签分组：3渲2基建、...、砍杀泛素材
            labels_str = [str(k) for k,v in mapping.items() if v==label]
            labels_str = ','.join(labels_str)
            for s,e in zip([0, 30, 60, 90], [0, 15, 30, 60]):  # 评分窗口：当天、15天前15天、30天前30天、60天前30天
                if label not in candi.keys():
                    candi[label] = collect(labels_str, 10, 400, s, e)  # TODO
                else:
                    candi[label] += collect(labels_str, 10, 400, s, e)  # TODO
            candi[label] = list(set(candi[label]) - set([41464, 41177, 40225, 41188, 38977, 40272, 40728, 40205, 39668, 38168, 41327, 41241, 35857, 39268]))
        return candi
    
    @staticmethod
    def get_images(labels, images_candi):
        """ 标签：素材下钻（样本） """
        cnt = Counter(labels)
        images = []
        for k,v in cnt.items():
            images += random.sample(images_candi[k], v) if len(images_candi[k]) > v else images_candi[k]
        return images
    
    def allocate(self, df1, df2):
        """ 标签：素材分配 """
        # -------大标签分组roi加权-------
        weight = df1['roi'] * df2['create_role_30_pay_sum'] * self.flex_coe  # 加权：大标签
        def normalization(data):
            _range = np.max(data) - np.min(data)
            return (data - np.min(data)) / _range
        
        weight = normalization(weight.astype(float))  # 归一化：大标签
        print("Normal归一化权重：", "\n", weight)
        labels = (pd.Series(range(len(self.flex_coe))) + 1).sample(n=self.plan_num, replace=True, weights=weight).values  # 采样：大标签
        print("标签：", labels)
        # -------小标签素材score下钻-------
        labels_images = self.get_images_candi(labels, self.mapping)  # 候选：素材
        print("标签下素材量：", "\n", labels_images)
        images = self.get_images(labels, labels_images) # 样本：素材
        
        return images, labels_images

In [34]:
label_ma = {'220':1,'117':2,'201':3,'114':4,'148':5,'115':5,'221':6,'159':7,'119':8,'157':8,'196':8,'116':9,'253':9}

### 标签素材

In [35]:
image_num = 0
flex_coe_tt = pd.Series([0,0,0,0,0,0,1,0,0])  # 自定义权重调节因子

In [36]:
# 3渲2基建、3D基建、大基建N、对比、荒岛剧情、兵种、录屏、小游戏、泛素材
image = ImageSec(label_ma, flex_coe_tt, image_num)
img, images = image.allocate(df_1, df_2)

0it [00:00, ?it/s]

Normal归一化权重： 
 0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    1.0
7    0.0
8    0.0
dtype: float64
标签： []
标签下素材量： 
 {}


In [37]:
for im in img:
    print(get_imageid([im]).values)

### 评分素材

In [38]:
def collect_roi():
    """ 评分筛选 """
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                   passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
          c.image_id,
          b.amount,
          b.new_role_money,
          b.new_role_money / b.amount AS roi,
          b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_stdata.st_lauch_report b LEFT JOIN db_data_ptom.ptom_plan c ON c.chl_user_id = b.channel_id 
            AND c.source_id = b.source_id
        WHERE
            b.tdate >= date( NOW() - INTERVAL 168 HOUR )  # 7天内3个点的好素材
            AND b.tdate_type = 'day' 
            AND b.media_id = 10
            AND b.game_id = 1001862
            AND b.amount >= 1000
            AND b.pay_role_user_num >= 1
            AND b.new_role_money >= 30
            AND (b.new_role_money / b.amount) >= 0.05
            AND c.platform = 1
            '''
    cur.execute(sql)
    result = as_pandas(cur)
    # 关闭链接
    cur.close()
    conn.close()
    return result['image_id'].values.tolist()

In [39]:
def collect_score():
    """ 评分筛选 """
    conn = connect(host='192.168.0.97', port=10000, auth_mechanism='PLAIN', user='hive',
                   password='Ycjh8FxiaoMtShZRd3-97%3hCEL0CK4ns1w', database='default')
    cursor = conn.cursor()
    sql_engine = 'set hive.execution.engine=tez'
    sql_queque = 'set tez.queue.name=offline'
    sql = 'select image_id,label_ids from dws.dws_image_score_d where media_id=10 and score>=600 and dt>=date_sub(current_date, 1) and dt<=date_sub(current_date, 0) group by image_id,label_ids'
    cursor.execute(sql_engine)
    cursor.execute(sql_queque)
    cursor.execute(sql)
    result = as_pandas(cursor)
    # 关闭链接
    cursor.close()
    conn.close()
    return result['image_id'].values.tolist()

In [40]:
def get_imageid_tp(image_ids):  
    """ 素材名：精准 """
    image_ids = [str(i) for i in image_ids]
    image_ids = ','.join(image_ids)
    
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            image_id,
            image_name,
            label_ids
        FROM
            db_data_ptom.ptom_image_info
        WHERE
            image_id IN ({})
            AND game_ids IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL )
    '''
    cur.execute(sql.format(image_ids))
    result = pd.read_sql(sql.format(image_ids), conn)
    # 关闭链接
    cur.close()
    conn.close()

    return result

In [41]:
images1 = collect_roi()  # 候选集1
images2 = collect_score()  # 候选集2
img_sc = get_imageid_tp(list(set(images1 + images2)))
img_sc

,image_id,image_name,label_ids
0,38987,QMZZ1703-横版LL(3D混剪3渲2）（TT）,220
1,39112,QMZZ1714-横版Z（3D复用3渲2）（TT）,220
2,40560,QMZZ1905-竖版Z（3D复用3渲2）（TT）,148
3,41327,QMZZ1958-竖版xy_头尾（3D复用3渲2）（MOD）（TT）,"199,220"
4,43861,QMZZ2291-横版LL（3D混剪3渲2）（TT）,220
5,41154,QMZZ1944-竖版LL（3D复用3渲2）（TT）,"220,236"
6,41497,QMZZ1980-横版Z（3D复用3渲2）（TT）,220
7,44038,QMZZ2308-竖版LMF(3D复用3渲2)(TT),220
8,44083,QMZZ2311-竖版LL（3渲2+MOD）帝国BGM(TT),220
9,44108,QMZZ2318-横版CHL（3D混剪3渲2）（TT）,220


In [42]:
img_sc = img_sc['image_id'].values.tolist()

# 新建计划
* 过年期间

## 采集函数

In [43]:
def get_game_id():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL 
    '''
    cur.execute(sql)
    result_df = cur.fetchall()
    cur.close()
    conn.close()
    return result_df


# 获取近期所有计划
def get_plan_info():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)
    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
    /*手动查询*/
        SELECT
                * 
            FROM
                db_ptom.ptom_third_plan p
            WHERE
                game_id IN ({})
                AND media_id = 10
                AND create_time>=date( NOW() - INTERVAL 1440 HOUR )
                AND create_time<= date(NOW())
                            AND plan_id >= (
                                select plan_id from db_ptom.ptom_plan
                                where create_time >= date( NOW() - INTERVAL 1440 HOUR )
                                and create_time <= date( NOW() - INTERVAL 1416 HOUR )
                                limit 1
                            )
    '''
    finalSql = sql.format(game_id)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result_df


# 获取image_id,label_ids
def get_image_info():
    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id,
            a.image_id,
            b.label_ids
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN
            db_data_ptom.ptom_image_info b
        on a.image_id = b.image_id
        WHERE
            a.game_id IN ( SELECT dev_game_id AS game_id FROM db_data.t_game_config WHERE game_id = 1112 AND dev_game_id IS NOT NULL ) 
            AND a.media_id = 10 
            AND a.create_time >= date( NOW() - INTERVAL 1440 HOUR )
        GROUP BY
            a.chl_user_id,
            a.source_id,
            a.image_id
    '''
    cur.execute(sql)
    result_df = pd.read_sql(sql, conn)
    cur.close()
    conn.close()
    return result_df


def get_launch_report():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/
        SELECT
            a.chl_user_id AS channel_id,
            a.source_id AS source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount as roi,
            b.pay_role_user_num / b.create_role_num as pay_rate
        FROM
            db_data_ptom.ptom_plan a
            LEFT JOIN db_stdata.st_lauch_report b ON a.chl_user_id = b.channel_id 
            AND a.source_id = b.source_id 
        WHERE
            a.create_time >= date( NOW() - INTERVAL 1440 HOUR )
            AND b.tdate >= date( NOW() - INTERVAL 1440 HOUR )
            AND b.tdate_type = 'day' 
            AND b.media_id = 10
            AND b.game_id IN ({})
            AND b.amount >= 500
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df


# 解析json
def get_plan_json(plan_info):
    plan_info.drop(['inventory_type', 'budget'], axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info.dropna(subset=['ad_info'], inplace=True)
    # 解析json
    plan_info['ad_info'] = plan_info['ad_info'].apply(json.loads)
    temp = plan_info['ad_info'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('ad_info', axis=1, inplace=True)
    temp = plan_info['audience'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('audience', axis=1, inplace=True)
    temp = plan_info['action'].apply(pd.Series)
    plan_info = pd.concat([plan_info, temp], axis=1)
    plan_info.drop('action', axis=1, inplace=True)
    plan_info.dropna(how='all', inplace=True, axis=1)
    plan_info = plan_info[['ad_account_id', 'game_id', 'channel_id', 'source_id',
                           'create_time', 'smart_bid_type', 'hide_if_exists', 'budget',
                           'delivery_range', 'adjust_cpa', 'inventory_type', 'hide_if_converted',
                           'flow_control_mode', 'schedule_time', 'cpa_bid', 'auto_extend_enabled',
                           'gender', 'city', 'platform', 'launch_price',
                           'retargeting_tags_exclude', 'interest_categories',
                           'ac', 'android_osv', 'location_type', 'retargeting_tags_include',
                           'ios_osv', 'interest_action_mode', 'age',
                           'action_categories', 'action_days', 'action_scene', 'deep_bid_type', 'roi_goal']]
    return plan_info


def get_all_data():
    # 读取历史数据

    plan_info = get_plan_info()
    image_info = get_image_info()
    launch_report = get_launch_report()

    plan_info = get_plan_json(plan_info)

    image_info.dropna(subset=['image_id'], inplace=True)
    image_info['image_id'] = image_info['image_id'].astype(int)
    return plan_info, image_info, launch_report

# 获取近期计划的运营数据
def get_now_plan_roi():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modelfenxi-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        SELECT
          b.channel_id,
          b.source_id,
            b.tdate,
            b.amount,
            b.new_role_money,
            b.new_role_money / b.amount AS roi,
            b.pay_role_user_num / b.create_role_num AS pay_rate 
        FROM
            db_stdata.st_lauch_report b
        WHERE
            b.tdate >= date( NOW() - INTERVAL 720 HOUR ) 
            AND b.tdate_type = 'day' 
            AND b.media_id = 10
            AND b.game_id IN ({}) 
            AND b.amount >= 500
            AND b.pay_role_user_num >= 1
            AND b.new_role_money >= 6
            AND (b.new_role_money / b.amount)>=0.0
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result_df = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    result_df['tdate'] = pd.to_datetime(result_df['tdate'])
    result_df = result_df.sort_values('tdate')
    result_df = result_df.drop_duplicates(['channel_id', 'source_id'], keep='first')

    return result_df


# 获取近期优化计划的创意数据
def get_creative():
    game_id = get_game_id()
    game_id = list(map(lambda x: x['game_id'], game_id))
    game_id = [str(i) for i in game_id]
    game_id = ','.join(game_id)

    conn = pymysql.connect(host='db-slave-modeltoufang-001.ch', port=3306, user='model_read',
                           passwd='aZftlm6PcFjN{DxIKOPr)BcutuJd<uYOC0P<8', db='db_ptom')
    cur = conn.cursor(cursor=pymysql.cursors.DictCursor)
    sql = '''
        /*手动查询*/ 
        SELECT
            b.chl_user_id AS channel_id,
            b.source_id,
            JSON_EXTRACT( a.creative_param, '$.ad_keywords' ) AS ad_keywords,
            JSON_EXTRACT( a.creative_param, '$.title_list' ) AS title_list,
            JSON_EXTRACT( a.creative_param, '$.third_industry_id' ) AS third_industry_id 
        FROM
            db_ptom.ptom_batch_ad_task a
            LEFT JOIN db_ptom.ptom_plan b ON a.plan_name = b.plan_name 
        WHERE
            a.media_id = 10 
            AND b.create_time >= date( NOW() - INTERVAL 1440 HOUR )    # 近60天
            AND a.game_id IN ({}) 
    '''
    finalSql = sql.format(game_id)
    cur.execute(finalSql)
    result = pd.read_sql(finalSql, conn)
    cur.close()
    conn.close()
    return result

In [44]:
plan_info, image_info, launch_report = get_all_data()
creative_info = get_creative()
now_plan_roi = get_now_plan_roi()

## 预处理

In [45]:
df = pd.merge(plan_info, image_info, on=['channel_id', 'source_id'], how='left')
df.dropna(subset=['image_id'], inplace=True)
df = pd.merge(df, launch_report, on=['channel_id', 'source_id'], how='left')
df.drop(df[df['tdate'].isna()].index, inplace=True)
df = df[df['amount'] >= 500]

df['platform'] = df['platform'].astype(str)
df['platform'] = df['platform'].map({"['ANDROID']": 1, "['IOS']": 2})
df['label'] = df.apply(lambda x: 1 if x.new_role_money / x.amount >= 0.01 else 0, axis=1)
df['ad_account_id'] = df['ad_account_id'].astype('int')
df['image_id'] = df['image_id'].astype('int')
df.rename(columns={'tdate': 'create_date'}, inplace=True)
df['create_date'] = pd.to_datetime(df['create_date'])
df['create_time'] = pd.to_datetime(df['create_time'])

df.drop(['budget', 'cpa_bid', 'channel_id', 'source_id', 'amount', 'roi', 'pay_rate',
         'new_role_money'], axis=1, inplace=True)

plan_info['create_time'] = pd.to_datetime(plan_info['create_time'])
plan_info_current = plan_info[plan_info['create_time'] >= pd.datetime.now() - pd.DateOffset(30)]

creative_info['title_list'] = creative_info['title_list'].fillna('[]')
creative_info['ad_keywords'] = creative_info['ad_keywords'].fillna('[]')
creative_info['title_list'] = creative_info['title_list'].apply(json.loads)
creative_info['ad_keywords'] = creative_info['ad_keywords'].apply(json.loads)
# =====
def doSubList(words):
    """ 过滤：关键词 """
    if '帝国时代' in words:
        words[words.index('帝国时代')]='帝王纪元'  ## TODO:修改
    else:
        words.append('帝王纪元')
    if len(words) > 20:
        words = words[:20]
    return words
creative_info['ad_keywords'] = creative_info['ad_keywords'].apply(doSubList)
# =====
now_plan_roi = pd.merge(now_plan_roi, creative_info, on=['channel_id', 'source_id'], how='left')

df_create = pd.merge(plan_info_current, image_info, on=['channel_id', 'source_id'], how='left')
df_create = pd.merge(df_create, now_plan_roi, on=['channel_id', 'source_id'], how='inner')

df_create['platform'] = df_create['platform'].astype(str)
df_create['platform'] = df_create['platform'].map({"['ANDROID']": 1, "['IOS']": 2})
df_create['platform'] = df_create['platform'].astype(int)

df_create.dropna(subset=['image_id'], inplace=True)
df_create['image_id'] = df_create['image_id'].astype(int)

df_create = df_create[df_create['platform'] == 1]

## 计划拼接
* 重点部分

In [46]:
df = df_create.copy()

#### 参数二

In [47]:
ad_account_id_group = np.array([10495,9047,9049])  ## 帝国纪元  39574, 39446, 40348, 40736, 40205, 41327

df_ = df[df['game_id'].isin([1001862, 1001681, 1001787])]  
image_id_group = df_['image_id'].values 
image_id_group = list(set(image_id_group))
print("在跑的素材池ID:", image_id_group, "\n", "在跑的素材池数量:", len(image_id_group))

image_id_group = list(set(img_sc + [44178,44083]))  # 加人工
image_id_group += [44038,44038,44038]
print("选择的素材ID:", image_id_group, "\n", "选择的素材数量:", len(image_id_group))

df = df[df['deep_bid_type'].isin(['DEEP_BID_DEFAULT', 'BID_PER_ACTION', 'ROI_COEFFICIENT'])]  ## TODO:纯付费无深度目标、深度目标每次付费、深度目标ROI系数
df = df[df['title_list'].apply(lambda x:True if isinstance(x, list) and len(x) > 1 else False)]

def plan_cut(df, ad_account_id_group, image_id_group):
    """ 闭包区分：计划要素 """
    plan = pd.DataFrame()
    ad_counts = np.array([0] * len(ad_account_id_group), dtype=int)

    # for image in sorted(image_id_group, key=lambda x: random.random()):
    for image in image_id_group:
        ad_account = ad_account_id_group[np.argmin(ad_counts)]
        temp = pd.DataFrame({'ad_account_id': [ad_account], 'image_id': [image]})  ## TODO:初始预算拔高一点反正有监控模型看着
        plan = plan.append(temp)
        ad_counts[ad_counts.argsort()[:1]] += 1

    plan = pd.DataFrame(np.repeat(plan.values, 1, axis=0), columns=plan.columns)

    plan['game_id'] = 1001862
    df = df[df['game_id'].isin([1001862])]
    print(df['deep_bid_type'].value_counts())

    # 选platform测试期默认[ANDROID]
    plan['platform'] = 1
    plan['platform'] = plan['platform'].apply(lambda x: '[ANDROID]' if x == 1 else '[IOS]')

    # 选android_osv
    count_df = pd.DataFrame(data=df['android_osv'].value_counts()).reset_index()
    count_df.columns = ['col', 'counts']
    if count_df.shape[0] != 0:
        count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
        plan['android_osv'] = plan['platform'].apply(lambda x: 'NONE' if x == '[IOS]' else
            np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])
    if count_df.shape[0] == 0:
        plan['android_osv'] = plan['platform'].apply(lambda x: 'NONE' if x == '[IOS]' else np.nan)

    # 选ios_osv
    count_df = pd.DataFrame(data=df['ios_osv'].value_counts()).reset_index()
    count_df.columns = ['col', 'counts']
    if count_df.shape[0] != 0:
        count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
        plan['ios_osv'] = plan['platform'].apply(lambda x: 'NONE' if x == '[ANDROID]' else
        np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0])
    if count_df.shape[0] == 0:
        plan['ios_osv'] = plan['platform'].apply(lambda x: 'NONE' if x == '[ANDROID]' else np.nan)

    # 选budget
    plan['budget'] = plan['ad_account_id'].apply(lambda x: 3000 if x == 10495 else 10000)

    # 选'ad_keywords', 'title_list', 'third_industry_id'  创意
    sample_df = df[['ad_keywords', 'title_list', 'third_industry_id']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)

    # 选'retargeting_tags_include','retargeting_tags_exclude'  人群包定向
    sample_df = df[['retargeting_tags_include', 'retargeting_tags_exclude']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)
    plan['retargeting_tags_exclude'] = plan['retargeting_tags_exclude'].apply(lambda x:[])  ## TODO:人群包排除（不排除）

    # 选'interest_action_mode','action_scene','action_days','action_categories' ,'interest_categories' 行为兴趣
    sample_df = df[['interest_action_mode', 'action_scene', 'action_days', 'action_categories', 'interest_categories']]
    sample_df = sample_df.sample(n=plan.shape[0], replace=True).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)
    plan['action_days'] = plan['action_days'].convert_dtypes()
    plan['interest_action_mode'] = 'RECOMMEND'  ## TODO:行为兴趣定向系统推荐

    # 选'deep_bid_type','roi_goal','smart_bid_type','adjust_cpa','cpa_bid', 'budget'出价方式
    sample_df = df[['deep_bid_type', 'roi_goal', 'smart_bid_type', 'adjust_cpa', 'cpa_bid']]
    sample_df = sample_df[sample_df['deep_bid_type'].isin(['DEEP_BID_DEFAULT', 'BID_PER_ACTION', 'ROI_COEFFICIENT'])]
    sample_df['weight'] = sample_df['deep_bid_type'].map({'BID_PER_ACTION':1, 'ROI_COEFFICIENT':0, 'DEEP_BID_DEFAULT':0})  ## TODO:ROI_COEFFICIENT跑法增加权重
    sample_df = sample_df.sample(n=plan.shape[0], weights=sample_df['weight']).reset_index(drop=True)
    plan = pd.concat([plan, sample_df], axis=1)
    plan['adjust_cpa'] = plan['adjust_cpa'].fillna(0)
    plan['adjust_cpa'] = plan['adjust_cpa'].astype(int)
    plan['roi_goal'] = plan['deep_bid_type'].apply(lambda x:0.023 if x=='ROI_COEFFICIENT' else np.nan)  ## TODO:roi跑法系数的调整

    # 部分定向
    cols = ['delivery_range', 'city', 'location_type', 'gender', 'age', 'ac', 'launch_price',
            'auto_extend_enabled', 'hide_if_exists', 'hide_if_converted', 'schedule_time', 'flow_control_mode']
    for col in cols:
        count_df = pd.DataFrame(data=df[col].value_counts()).reset_index()
        count_df.columns = ['col', 'counts']
        count_df['pct'] = count_df['counts'] / count_df['counts'].sum()
        plan[col] = plan.apply(lambda x: np.random.choice(count_df['col'].values, 1, p=count_df['pct'].values)[0],
                            axis=1)
    plan['gender'] = 'GENDER_MALE'  ## TODO:性别男

    # 年龄定向写死
    plan['age'] = 'AGE_BETWEEN_24_30,AGE_BETWEEN_31_40,AGE_BETWEEN_41_49'  ## TODO:AGE_BETWEEN_18_23,
    plan['age'] = plan['age'].apply(lambda x: x.split(','))

    # 选inventory_type 创意投放位置（分跑法加权版位）
    count_df_default = pd.DataFrame(data=df[df['deep_bid_type']=='DEEP_BID_DEFAULT']['inventory_type'].value_counts()).reset_index()
    count_df_action = pd.DataFrame(data=df[df['deep_bid_type']=='BID_PER_ACTION']['inventory_type'].value_counts()).reset_index()
    count_df_roi = pd.DataFrame(data=df[df['deep_bid_type']=='ROI_COEFFICIENT']['inventory_type'].value_counts()).reset_index()

    count_df_default.columns, count_df_action.columns, count_df_roi.columns = ['col', 'counts'], ['col', 'counts'], ['col', 'counts']
    count_df_default['col'] = count_df_default['col'].apply(lambda x: list(filter(None, x)))
    count_df_action['col'] = count_df_action['col'].apply(lambda x: list(filter(None, x)))
    count_df_roi['col'] = count_df_roi['col'].apply(lambda x: list(filter(None, x)))

    count_df_default['pct'] = count_df_default['counts'] / count_df_default['counts'].sum()
    count_df_action['pct'] = count_df_action['counts'] / count_df_action['counts'].sum()
    count_df_roi['pct'] = count_df_roi['counts'] / count_df_roi['counts'].sum()

    mapping = {'DEEP_BID_DEFAULT':count_df_default, 'BID_PER_ACTION':count_df_action, 'ROI_COEFFICIENT':count_df_roi}
    plan['inventory_type'] = plan['deep_bid_type'].apply(lambda x: np.random.choice(mapping[x]['col'].values, 1, p=mapping[x]['pct'].values)[0])
    plan['inventory_type'] = plan['inventory_type'].apply(lambda x: random.choice([["INVENTORY_AWEME_FEED","INVENTORY_FEED"], random.choice([["INVENTORY_AWEME_FEED","INVENTORY_FEED"], ["INVENTORY_AWEME_FEED","INVENTORY_FEED"]])]))  ## TODO:多版位 ,"INVENTORY_FEED","INVENTORY_HOTSOON_FEED","INVENTORY_TOMATO_NOVEL","INVENTORY_UNION_SLOT"
    # plan['inventory_type'] = plan[['image_id', 'inventory_type']].apply(lambda x: random.choice([random.choice([['INVENTORY_TOMATO_NOVEL','INVENTORY_AWEME_FEED','INVENTORY_FEED'], ['INVENTORY_AWEME_FEED']]), ['INVENTORY_AWEME_FEED','INVENTORY_FEED']]) if x.image_id in image_id_group_1 else x.inventory_type, axis=1)  ## TODO:双版位
    # plan['inventory_type'] = plan['inventory_type'].apply(lambda x: ['INVENTORY_AWEME_FEED','INVENTORY_FEED'])  ## TODO:双版位不变
    plan['create_time'] = pd.to_datetime(pd.datetime.now())
    plan['create_date'] = pd.to_datetime(pd.datetime.now().date())

    # plan['cpa_bid'] = plan[['ad_account_id','deep_bid_type','cpa_bid']].apply(lambda x: 588.88 if x.ad_account_id == 10495 else 888.88, axis=1)  # and x.deep_bid_type == 'BID_PER_ACTION' 

    # plan['plan_auto_task_id'] = plan['image_id'].apply(lambda x: "10998,11002,12066" if x not in image_id_group_2 else "10998,11002")
    plan['plan_auto_task_id'] = "10998,11002,12066"
    return plan

plan = plan_cut(df, ad_account_id_group, image_id_group)
plan = plan.reset_index(drop=True)

在跑的素材池ID: [44038, 44039, 41993, 43273, 43280, 32274, 32016, 44053, 39446, 42518, 41497, 40736, 42535, 38443, 43564, 42289, 44083, 41781, 43319, 37944, 43070, 38987, 44108, 43854, 44117, 42325, 43861, 43096, 42332, 44126, 43619, 42601, 43114, 43625, 43883, 40560, 32124, 40578, 39559, 43657, 44170, 43660, 44176, 43154, 43667, 44178, 43411, 39574, 41365, 41368, 42394, 40094, 43681, 43176, 43689, 44202, 43444, 41154, 42183, 39112, 38865, 40404, 43480, 37850, 42463, 43749, 43239, 43253, 43258, 43515, 37886] 
 在跑的素材池数量: 71
选择的素材ID: [41154, 44038, 39112, 38987, 44108, 41327, 40560, 44178, 44083, 43861, 41497, 44038, 44038, 44038] 
 选择的素材数量: 14
BID_PER_ACTION    173
Name: deep_bid_type, dtype: int64


## 函数

In [48]:
def doSubStr(row):
    """ 过滤：推广文案 """
    dic = {1001862:'帝王'}
    x = row.copy()
    res = x['title_list']

    if '罗马单机版' in res:
        res = re.sub('罗马单机版', dic[x['game_id']], res)
    if '罗马单机加强版' in res:
        res = re.sub('罗马单机加强版', dic[x['game_id']], res)
    if '罗马单机时代版' in res:
        res = re.sub('罗马单机时代版', dic[x['game_id']], res)
    if '领主荣耀' in res:
        res = re.sub('领主荣耀', dic[x['game_id']], res)
    if '帝国单机版' in res:
        res = re.sub('帝国单机版', dic[x['game_id']], res)
    if '帝国时代单机版' in res:
        res = re.sub('帝国时代单机版', dic[x['game_id']], res)
    if '纯单机版帝国' in res:
        res = re.sub('纯单机版帝国', dic[x['game_id']], res)
    if '帝国高清复刻版' in res:
        res = re.sub('帝国高清复刻版', dic[x['game_id']], res)
    if '帝国' in res:
        res = re.sub('帝国', '帝王', res)
    return res

def get_ad_create(plan_result):
    ad_info = []
    for i in range(plan_result.shape[0]):
        ad_info.append(json.loads(plan_result.iloc[i].to_json()))
    open_api_url_prefix = "https://ptom.caohua.com/"
    # open_api_url_prefix = "http://ptom-pre.caohua.com/"
    uri = "model/generationPlanBatchTask"
    url = open_api_url_prefix + uri
    params = {
        "secretkey": "abc2018!@**@888",
        "mediaId": 10
    }
    rsp = requests.post(url, json=ad_info, params=params)
    rsp_data = rsp.json()
    print('[INFO]结束....')
    return rsp_data

## 后处理

In [49]:
image_info.dropna(subset=['image_id'], inplace=True)
image_info['image_id'] = image_info['image_id'].astype(int)
plan_create = pd.merge(plan, image_info[['image_id', 'label_ids']].drop_duplicates(), on='image_id',
                       how='left')

plan_result = plan_create.drop(['create_time', 'create_date', 'label_ids'], axis=1)
plan_result['convertIndex'] = plan_result['deep_bid_type'].apply(lambda x:14 if x == 'ROI_COEFFICIENT' else 13)

plan_result['budget'] = plan_result.apply(lambda x: x.budget if x.budget >= x.cpa_bid else x.cpa_bid, axis=1)
plan_result['budget'] = plan_result['budget'].apply(np.ceil)

# # 过滤武汉
# plan_result['city'] = plan_result['city'].apply(
#     lambda x: x if x != [] else [11, 12, 13, 14, 15, 21, 22, 23, 31, 32, 33, 34, 35, 36, 37, 41, 420200, 420300,
#                                  420500, 420600, 420700, 420800, 420900, 421000, 421100, 421200, 421300, 422800,
#                                  429004, 429005, 429006, 429021, 43, 44, 45, 46, 50, 51, 52, 53, 54, 61, 62, 63, 64,
#                                  65, 71, 81, 82])
# plan_result['schedule_time'] = '0'*48*3 + '1'*48*2 + '111111111111111111111111111111111111111111111111'*2
plan_result['web_url'] = 'https://www.chengzijianzhan.com/tetris/page/7063028891002175495/'  # 帝国纪元:单机加强版
# plan_result['operation'] = 'disable'
plan_result['op_id'] = 13268
plan_result['district'] = 'CITY'
plan_result['flag'] = 'SSR'
plan_result['budget'] = plan_result['budget'].astype(int)
plan_result['image_id'] = plan_result['image_id'].astype(int)

# 文案过滤
plan_result['title_list'] = plan_result['title_list'].map(str)
plan_result['title_list'] = plan_result.apply(doSubStr, axis=1)

plan_result = plan_result[plan_result['game_id']==1001862]
plan_result.to_csv('./plan_result.csv', index=0)  # 保存创建日志

plan_result

,ad_account_id,image_id,game_id,platform,android_osv,ios_osv,budget,ad_keywords,title_list,third_industry_id,...,hide_if_converted,schedule_time,flow_control_mode,inventory_type,plan_auto_task_id,convertIndex,web_url,op_id,district,flag
0,10495,41154,1001862,[ANDROID],8.0,NONE,3000,"[手游, SLG, 策略, 战争策略, 策略手游, 最强王者, 我的使命, 红警, 帝国文明...","[{'title': '开局只有一个农民，从荒岛到王国，你需要多久？', 'creative...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
1,9047,44038,1001862,[ANDROID],8.0,NONE,10000,"[手游, SLG, 策略, 战争策略, 策略手游, 最强王者, 我的使命, 红警, 帝国文明...","[{'title': '开局只有一个农民，从荒岛到王国，你需要多久？', 'creative...",19131201,...,AD,0000000000000000000000000000000000000000000000...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
2,9049,39112,1001862,[ANDROID],8.0,NONE,10000,"[手游, SLG, 策略, 战争策略, 策略手游, 最强王者, 我的使命, 红警, 帝国文明...","[{'title': '开局只有一个农民，从荒岛到王国，你需要多久？', 'creative...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
3,10495,38987,1001862,[ANDROID],10.1,NONE,3000,"[帝国, 帝王纪元, 帝国纪元, 帝国单机版, 罗马帝国, 帝国战魂, 战争, SLG, 策...","[{'title': '苦等20年，手游终于出了！满满的儿时回忆，老玩家已感动哭！', 'c...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
4,9047,44108,1001862,[ANDROID],8.0,NONE,10000,"[手游, SLG, 策略, 战争策略, 策略手游, 最强王者, 我的使命, 红警, 帝国文明...","[{'title': '开局只有一个农民，从荒岛到王国，你需要多久？', 'creative...",19131201,...,APP,1111111111111111111111111111111111111111111110...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
5,9049,41327,1001862,[ANDROID],8.0,NONE,10000,"[帝国, 帝王纪元, 帝国纪元, 罗马时代, 文明, COK, 文明与征服, 列王的纷争, ...","[{'title': '三大文明碰撞，伐木、采矿、掠夺、建造，多种发展路线供你选择！', '...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
6,10495,40560,1001862,[ANDROID],8.0,NONE,3000,"[帝国, SLG, 策略, 部落冲突, 武将, 兵种, 王国纪元, 攻城略地, 战争, RP...","[{'title': '古巴比伦、欧洲、亚洲三选一，攻城占地，开启你的王国王朝！', 'cr...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
7,9047,44178,1001862,[ANDROID],8.0,NONE,10000,"[策略手游, 手游, 沙盘游戏, 帝王纪元, SLG, 角色扮演, 网易, 腾讯, 策略, ...","[{'title': ' 从开局4农民到征战四方，建立自己的帝王文明，你需要多久？', 'c...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
8,9049,44083,1001862,[ANDROID],10.1,NONE,10000,"[游戏, 手游, 对战, 军事手游, 三国志, 三国志策略版, 万国觉醒, 传奇, 深蓝战舰...","[{'title': '小时候玩的游戏终于出手游了,采集伐木，挖矿，打造你的帝王!', 'c...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR
9,10495,43861,1001862,[ANDROID],8.0,NONE,3000,"[手游, SLG, 策略, 战争策略, 策略手游, 最强王者, 我的使命, 红警, 帝国文明...","[{'title': '开局只有一个农民，从荒岛到王国，你需要多久？', 'creative...",19131201,...,APP,1111111111111111111111111111111111111111111111...,FLOW_CONTROL_MODE_FAST,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]","10998,11002,12066",13,https://www.chengzijianzhan.com/tetris/page/70...,13268,CITY,SSR


In [50]:
plan_result[['deep_bid_type','inventory_type','cpa_bid']]

,deep_bid_type,inventory_type,cpa_bid
0,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",588.88
1,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
2,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
3,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",588.88
4,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
5,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
6,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",588.88
7,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
8,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",1588.88
9,BID_PER_ACTION,"[INVENTORY_AWEME_FEED, INVENTORY_FEED]",588.88


In [51]:
img = get_imageid_tp(plan_result['image_id'])
img

,image_id,image_name,label_ids
0,38987,QMZZ1703-横版LL(3D混剪3渲2）（TT）,220
1,39112,QMZZ1714-横版Z（3D复用3渲2）（TT）,220
2,40560,QMZZ1905-竖版Z（3D复用3渲2）（TT）,148
3,41327,QMZZ1958-竖版xy_头尾（3D复用3渲2）（MOD）（TT）,"199,220"
4,43861,QMZZ2291-横版LL（3D混剪3渲2）（TT）,220
5,41154,QMZZ1944-竖版LL（3D复用3渲2）（TT）,"220,236"
6,41497,QMZZ1980-横版Z（3D复用3渲2）（TT）,220
7,44038,QMZZ2308-竖版LMF(3D复用3渲2)(TT),220
8,44083,QMZZ2311-竖版LL（3渲2+MOD）帝国BGM(TT),220
9,44108,QMZZ2318-横版CHL（3D混剪3渲2）（TT）,220


## 结束

In [53]:
# rsp_data = get_ad_create(plan_result)  # 历史高R优质素材  自主分配

[INFO]结束....
